In [1]:
#!pip install sagemaker
import sagemaker
import boto3
from sagemaker.amazon.amazon_estimator import get_image_uri
from sagemaker.session import s3_input,Session

ModuleNotFoundError: No module named 'sagemaker'

In [22]:
buck='titleapp1'
my_reg=boto3.session.Session()
my_reg
s3.create_bucket(Bucket=buck)

{'ResponseMetadata': {'RequestId': 'GJMWYGBPAZTBH4HW',
  'HostId': 'q4uCC2uSgEhKyXbOyR/Q4/VU15zK1fcPv2sepp7rxdIclb+3tsKBOGgr0fYuEBcEvsNPw79ZvJVIStuNGwSkN6sbOF2XFZNhOobsEvCm+xY=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'q4uCC2uSgEhKyXbOyR/Q4/VU15zK1fcPv2sepp7rxdIclb+3tsKBOGgr0fYuEBcEvsNPw79ZvJVIStuNGwSkN6sbOF2XFZNhOobsEvCm+xY=',
   'x-amz-request-id': 'GJMWYGBPAZTBH4HW',
   'date': 'Thu, 09 May 2024 11:24:07 GMT',
   'location': '/titleapp1',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'Location': '/titleapp1'}

In [16]:
s3=boto3.client('s3')
try:
    if my_reg=='us-east-1':
        s3.create_bucket(Bucket=buck, LocationConstraint='us-east-1')
    print('success')
except Exception as e:
    print(e)

success


In [32]:
pre='xgboost'
out=f's3://{buck}/{pre}/output'

In [23]:
import pandas as pd
import urllib
try:
    urllib.request.urlretrieve ("https://d1.awsstatic.com/tmt/build-train-deploy-machine-learning-model-sagemaker/bank_clean.27f01fbbdf43271788427f3682996ae29ceca05d.csv", "bank_clean.csv")
    print('Success: downloaded bank_clean.csv.')
except Exception as e:
    print('Data load error: ',e)

try:
    model_data = pd.read_csv('./bank_clean.csv',index_col=0)
    print('Success: Data loaded into dataframe.')
except Exception as e:
    print('Data load error: ',e)

Success: downloaded bank_clean.csv.
Success: Data loaded into dataframe.


In [28]:
#! pip install -U scikit-learn scipy matplotlib
from sklearn.model_selection import train_test_split

train_data,test_data=train_test_split(model_data,test_size=0.3,random_state=42)
train_data.shape,test_data.shape

((28831, 61), (12357, 61))

In [35]:
import os
pd.concat([train_data['y_yes'], train_data.drop(['y_no', 'y_yes'], 
                                                axis=1)], 
                                                axis=1).to_csv('train.csv', index=False, header=False)
boto3.Session().resource('s3').Bucket(buck).Object(os.path.join(pre, 'train/train.csv')).upload_file('train.csv')
s3_input_train = sagemaker.TrainingInput(s3_data='s3://{}/{}/train'.format(buck, pre), content_type='csv')

pd.concat([test_data['y_yes'], test_data.drop(['y_no', 'y_yes'], axis=1)], axis=1).to_csv('test.csv', index=False, header=False)
boto3.Session().resource('s3').Bucket(buck).Object(os.path.join(pre, 'test/test.csv')).upload_file('test.csv')
s3_input_test = sagemaker.TrainingInput(s3_data='s3://{}/{}/test'.format(buck, pre), content_type='csv')

In [37]:
container =get_image_uri(boto3.Session().region_name,'xgboost',repo_version='1.0-1')
hyperparameters = {
        "max_depth":"5",
        "eta":"0.2",
        "gamma":"4",
        "min_child_weight":"6",
        "subsample":"0.7",
        "objective":"binary:logistic",
        "num_round":50
        }

The method get_image_uri has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


In [39]:
 estimator = sagemaker.estimator.Estimator(image_uri=container, 
                                          hyperparameters=hyperparameters,
                                          role=sagemaker.get_execution_role(),
                                          train_instance_count=1, 
                                          train_instance_type='ml.m5.2xlarge', 
                                          train_volume_size=5, # 5 GB 
                                          output_path=out,
                                          train_use_spot_instances=True,
                                          train_max_run=300,
                                          train_max_wait=600)

train_instance_count has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_instance_type has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_max_run has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_use_spot_instances has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_max_wait has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_volume_size has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


In [42]:
estimator.fit({'train':s3_input_train,'validation':s3_input_test})

INFO:sagemaker:Creating training-job with name: sagemaker-xgboost-2024-05-09-13-51-21-315


2024-05-09 13:51:21 Starting - Starting the training job...
2024-05-09 13:51:36 Starting - Preparing the instances for training...
2024-05-09 13:52:03 Downloading - Downloading input data...
2024-05-09 13:52:44 Training - Training image download completed. Training in progress....
2024-05-09 13:53:20 Uploading - Uploading generated training model
2024-05-09 13:53:20 Completed - Training job completed
[2024-05-09 13:53:05.255 ip-10-0-81-118.ec2.internal:7 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
INFO:sagemaker-containers:Imported framework sagemaker_xgboost_container.training
INFO:sagemaker-containers:Failed to parse hyperparameter objective value binary:logistic to Json.
Returning the value itself
INFO:sagemaker-containers:No GPUs detected (normal if no gpus installed)
INFO:sagemaker_xgboost_container.training:Running XGBoost Sagemaker in algorithm mode
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determine